In [ ]:
#study 用

# import datasets

# knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
# docs=[doc for doc in knowledge_base]
# docs[0]

In [19]:
import glob
knowledge_base=[]
read_files = glob.glob("./数据分析教程/*.txt")
for f in read_files:
    with open(f,"r") as fp:
        knowledge_base.append({'text':fp.read(),'source':f})

In [21]:
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from tqdm import tqdm

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed.append(new_doc)

Splitting documents...


100%|██████████| 27/27 [00:01<00:00, 25.05it/s]


In [24]:

print("Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Embedding documents... This should take a few minutes (5 minutes on MacBook with M1 Pro)


C:\Users\aesc\AppData\Local\Temp\ipykernel_14140\1302164431.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


In [ ]:
#使用本地vectordb
from langchain.vectorstores import FAISS
# vectordb=FAISS.load_local("./vectordb",embeddings=HuggingFaceEmbeddings(model_name="thenlper/gte-small"),allow_dangerous_deserialization=True)
vectordb.save_local("vectordb")

In [10]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

In [ ]:
from huggingface_hub import login


In [12]:
from smolagents import HfApiModel, ToolCallingAgent,OpenAIServerModel
model=OpenAIServerModel(model_id='qwen-plus',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
retriever_tool = RetrieverTool(vectordb)
agent = ToolCallingAgent(tools=[retriever_tool], model=model)

In [13]:
agent_output = agent.run("How can I push a model to the Hub?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub?                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen-plus ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'push model to Hub'}                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
--push_to_hub
```===== Document 1 =====
```py
>>> trainer.push_to_hub()
```
</pt>
<tf>
Share a model to the Hub with |`PushToHubCallback`]. In the |`PushToHubCallback`] function, add:

- An output directory for your model.
- A tokenizer.
- The `hub_model_id`, which is your Hub username and model name.

```py
>>> from transformers import PushToHubCallback

>>> push_to_hub_callback = PushToHubCallback(
...     output_dir="./your_model_save_path", tokenizer=tokenizer, hub_model_id="your-username/my-awesome-model"
... )
```===== Document 2 =====
# Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)
```

### .

By using `push_to_hub` **you evaluate, record a replay, generate a model card of your agent and push it to the 
Hub**.===== Document 3 =====
processor.push_to_hub(hub_model_id)
trainer.push_to_hub(**kwargs)
```

# 4. Inference

Now comes the exciting part, using our fine-tuned model! In this section, we'll show how you can load your model 
from the hub and use it for inference.===== Document 4 =====
Finally, if you want, you can push your model up to the hub. Here, we'll push it up if you specified 
`push_to_hub=True` in the training configuration. Note that in order to push to hub, you'll have to have git-lfs 
installed and be logged into your Hugging Face account (which can be done via `huggingface-cli login`).

```python
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'beans',
    "tags": |'image-classification'],
}===== Document 5 =====
The `push_to_hub=True` argument will allow us to push the model to the Hub after training; you'll find the 
repository under your user profile in the location defined by `output_dir`. Note that you can specify the name of 
the repository you want to push to with the `hub_model_id` argument (in particular, you will have to use this 
argument to push to an organization). For instance, when we pushed the model to the |`huggingface-course` 
organization](https://huggingface.co/huggingface-course), we added 
`hub_model_id="huggingface-course/mt5-finetuned-amazon-en-es"` to `Seq2SeqTrainingArguments`.===== Document 6 =====
{/if}

At a lower level, accessing the Model Hub can be done directly on models, tokenizers, and configuration objects via
their `push_to_hub()` method. This method takes care of both the repository creation and pushing the model and 
tokenizer files directly to the repository. No manual handling is required, unlike with the API we'll see below.

To get an idea of how it works, let's first initialize a model and a tokenizer:

{#if fw === 'pt'}
```py
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"

[Step 1: Duration 1.65 seconds| Input tokens: 1,199 | Output tokens: 20]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'To push a model to the Hub, you can use the            │
│ `push_to_hub` method from the trainer object or the `PushToHubCallback` function. Alternatively, you can use    │
│ the `api.upload_folder` method or set `push_to_hub=True` in the training configuration. Lastly, you can access  │
│ the Model Hub directly on models, tokenizers, and configuration objects via their `push_to_hub()` method.'}     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: To push a model to the Hub, you can use the `push_to_hub` method from the trainer object or the 
`PushToHubCallback` function. Alternatively, you can use the `api.upload_folder` method or set `push_to_hub=True` 
in the training configuration. Lastly, you can access the Model Hub directly on models, tokenizers, and 
configuration objects via their `push_to_hub()` method.

[Step 2: Duration 27.25 seconds| Input tokens: 3,205 | Output tokens: 465]

Final output:
To push a model to the Hub, you can use the `push_to_hub` method from the trainer object or the `PushToHubCallback` function. Alternatively, you can use the `api.upload_folder` method or set `push_to_hub=True` in the training configuration. Lastly, you can access the Model Hub directly on models, tokenizers, and configuration objects via their `push_to_hub()` method.


In [ ]:
new_docs = text_splitter.split_documents([source_docs[0]])

In [ ]:
knowledge_base[0]